# Manually rebuilding the stiffness matrix

Hermite dofs at the boundary are also set to zero by `DirichletBC.apply()`: we manually find those and reset the relevant rows of the mass matrix.

In the process of writing `find_hermite_boundary_dofs()` and `apply_dirichlet_hermite()` it was useful to look into the code for `dolfin/fem/DirichletBC.cpp, compute_bc_pointwise()`. Also there is a [bug](https://bugs.launchpad.net/dolfin/+bug/1063868) in the automatically generated documentation which affects [`GenericMatrix.set()`](https://fenicsproject.org/olddocs/dolfin/2016.2.0/python/programmers-reference/cpp/la/GenericMatrix.html?highlight=genericmatrix#dolfin.cpp.la.GenericMatrix.set) below. Only one function with three parameters is actually exposed by SWIG and the second and last are actually the indices of the rows and columns to modify. Finally, modifying a sparse (AIJ) PETSc matrix after assembly raises [error #63 (out of bounds)](https://www.mcs.anl.gov/petsc/petsc-current/include/petscerror.h.html) for insertions violating the sparsity pattern. In order to fix this we can use `setOption(MAT_NEW_NONZERO_ALLOCATION_ERR, PETSc_FALSE)`, see below.

## To do

* Check whether `apply_dirichlet_hermite()` breaks symmetries and try to fix that.
* Implement `apply_neumann_hermite()` for dim > 1.

**Note:** A better approach would be to "fix" `DirichletBC`: It seems like I need to define a special dofmap for Hermite elements which in `dofmap.cell_dofs()` returns only the Lagrange dofs (?). Won't this break lots of other things? Instead I could try to construct a copy of the function space with the right dofmap for the BCs and pass it to DirichletBC's constructor. What is the right approach? **See how it's done for periodic boundary conditions.**
What about sub dofmaps? See `DirichletBC::compute_bc_pointwise|topological|whatever()`.

In [ ]:
from dolfin import *
import FIAT
import ffc
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline
from petsc4py import PETSc
from utils import Msg

def __nbinit__():
    """ Initialisation for nbimporter. """

    # HACK: These constants taken from PETSc's doc.
    # http://www.mcs.anl.gov/petsc/petsc-current/docs/manualpages/Mat/MatOption.html
    PETSc.MAT_NEW_NONZERO_ALLOCATION_ERR = 19
    PETSc.PETSc_FALSE = 0
    PETSc.PETSc_TRUE = 1

    global NeumannBC
    NeumannBC = DirichletBC

    global __all__
    __all__ = ['apply_dirichlet_hermite', 'apply_neumann_hermite', 
               'plot_dofs', 'plot_hermite_dofs',
               'NeumannBC']
__nbinit__()

In [ ]:
def find_hermite_dofs(V:FunctionSpace) -> np.ndarray:
    """ Returns the global indices of Hermite dofs in V.

    Arguments
    ---------
        V: Any FunctionSpace. If it has Hermite (or DKT) elements, then
           the indices of those dofs associated to PointDerivatives will
           be returned.
    Returns
    -------
        Global indices of hermite dofs.
    """
    tdim = V.element().topological_dimension()
    e = ffc.fiatinterface.create_element(V.ufl_element())
    
    # This mask filters out the Hermite dofs from the list of dofs of a cell:
    mask = np.array(list(map(lambda f: isinstance(f, FIAT.functional.PointDerivative),
                             e.dual_basis())))

    # Compute the set of hermite dofs
    hermite_dofs = set()
    dm = V.dofmap()
    for i in range(V.mesh().num_cells()):
        hermite_dofs = hermite_dofs.union(set(dm.cell_dofs(i)[mask]))
    return np.array(list(hermite_dofs), dtype=np.intc)

In [ ]:
def find_hermite_boundary_dofs(bc:DirichletBC) -> np.ndarray:
    """ Returns the Hermite dofs at the boundary as defined in the argument.
    
    Arguments
    ---------
        bc: Boundary condition implementing get_boundary_values()

    Returns
    -------
        Global indices of hermite dofs on the boundary.
    """
    hermite_dofs = find_hermite_dofs(bc.function_space())
    # This is an easy way of retrieving the ids of dofs at the boundary:
    boundary_dofs = list(bc.get_boundary_values())

    return np.array(list(filter(lambda x: x in boundary_dofs, hermite_dofs)),
                    dtype=np.intc)

From the [mailing list](http://lists.mcs.anl.gov/pipermail/petsc-users/2012-February/012242.html):

    "Preallocation routines now automatically set MAT_NEW_NONZERO_ALLOCATION_ERR,
     if you intentionally preallocate less than necessary then use
     MatSetOption(mat,MAT_NEW_NONZERO_ALLOCATION_ERR,PETSC_FALSE) to disable the
     error generation"

See: [PETScBool](http://www.mcs.anl.gov/petsc/petsc-current/docs/manualpages/Sys/PetscBool.html), [PETSc MatOption](http://www.mcs.anl.gov/petsc/petsc-current/docs/manualpages/Mat/MatOption.html), [PETSc error codes](https://www.mcs.anl.gov/petsc/petsc-current/include/petscerror.h.html).

In [ ]:
def apply_dirichlet_hermite(A:Matrix, b:Vector, bc:DirichletBC):
    """ Applies DirichletBC bc to A, b in place but fixes the rows
    corresponding to Hermite dofs.
    
    Arguments:
    ----------
        A: assembled mass matrix, before applying bc. (I/O)
        b: assembled right hand side, before applying bc. (I/O)
        bc: DirichletBC
    Returns:
    --------
        Nothing. Arguments are modified in place.
    """
    global parameters
    assert isinstance(bc, DirichletBC),\
           "We only know how to manage Dirichlet BCs"
    warning("This won't work in parallel!")
    
    rows = find_hermite_boundary_dofs(bc)
    cols = np.arange(A.size(1), dtype=np.intc)
    saved_block = np.empty((rows.size, cols.size), dtype=np.float)
    A.get(saved_block, rows, cols)
    
    saved_vec = b[rows].copy()
    
    # Applying BCs messes things up for Hermite dofs
    # NOTE: DirichletBC.apply() does not enforce symmetry of A
    with Msg("Applying"):
        bc.apply(A, b)
    
    # So we fix them now:
    with Msg("Restoring RHS"):
        b[rows] = saved_vec
    M = as_backend_type(A)
    if isinstance(M, dolfin.cpp.la.PETScMatrix):
        M.mat().setOption(PETSc.MAT_NEW_NONZERO_ALLOCATION_ERR,
                          PETSc.PETSc_FALSE)
        
    # This will be slow:
    with Msg("Restoring LHS"):
        M.set(saved_block, rows, cols)
    # No need to reset the columns: bc.apply() does not enforce symmetry
    # (dolfin.assemble_system() does, though)
    with Msg("Applying"):
        M.apply('insert')
    
    # FIXME: should I set MAT_NEW_NONZERO_ALLOCATION_ERR back to TRUE?

# THIS NEEDS TESTING!

I need to check the symmetrization in `apply_neumann_hermite()`:

In [ ]:
def apply_neumann_hermite(A:Matrix, b:Vector, bc):
    """ Applies an *essential* NeumannBC bc to A, b in place by 
    setting rows for Hermite dofs.

    WARNING: it is essential that the BC *be essential* in the
    weak formulation of the PDE. :P
    
    Arguments:
    ----------
        A: assembled mass matrix, before applying bc
        b: assembled right hand side, before applying bc
        bc: NeumannBC (alias for DirichletBC) encoding the value
            of the normal derivative
    """
    if bc is None:     # HACK (what for?)
        return
    global parameters
    assert isinstance(bc, NeumannBC),\
           "We only know how to manage NeumannBCs"
    assert bc.function_space().element().geometric_dimension() == 1,\
           "FIXME: I can only manage 1 dimensional normal derivatives."

    warning("This won't work in parallel!")
    warning("Probably this won't work at all!")
    
    rows = find_hermite_boundary_dofs(bc)
    
    vals = bc.get_boundary_values()  # This is a dict
    for dof in rows:
        b[dof] = vals[dof]

    ncols = A.size(1)
    nrows = rows.size
    block = np.zeros((nrows, ncols))
    block[range(nrows),rows] = 1.
    
    cols = np.arange(ncols, dtype=np.intc)
    M = as_backend_type(A)
    if isinstance(M, dolfin.cpp.la.PETScMatrix):
        M.mat().setOption(PETSc.MAT_NEW_NONZERO_ALLOCATION_ERR,
                          PETSc.PETSc_FALSE)

    # This will be slow:
    with Msg("Applying Neumann BC"):
        M.set(block, rows, cols)
        M.apply('insert')

    # FIXME: should I set MAT_NEW_NONZERO_ALLOCATION_ERR back to TRUE?
    
    # Enforce symmetry by Gaussian elimination
    with Msg("Symmetrizing"):
        # Slow! Should be done at the element level
        lhs_column = np.empty((A.size(0), 1), dtype=np.float)
        all_rows = np.arange(A.size(0), dtype=np.intc)
        dofs = np.array(rows, dtype=np.intc).reshape((-1,1))
        for dof in dofs:
            M.get(lhs_column, all_rows, dof)
            save = b[dof]
            b -= lhs_column*save
            b[dof] = save
            lhs_column.fill(0.)
            lhs_column[dof] = 1.
            M.set(lhs_column, all_rows, dof)
            M.apply('insert')

In [ ]:
def plot_dofs(V:FunctionSpace, dofs:list, **kwargs):
    """ Plots the mesh and the coordinates of dofs in it
    with colored dots.
    
    Arguments
    ---------
        V: FunctionSpace of dimension 1,2 or 3
        dofs: list of indices.
        **kwargs: anything that matplotlib accepts.
    """
    tdim = V.mesh().topology().dim()
    all_dofs = zip(V.dofmap().dofs(), 
                   V.tabulate_dof_coordinates().reshape((-1, tdim)))
    dofs_coordinates = np.array([dof[1] for dof in 
                                 filter(lambda p: p[0] in dofs, all_dofs)])
    plot(V.mesh())

    if dofs_coordinates.size == 0:
        return

    kwargs.setdefault('linewidths', 0)
    kwargs.setdefault('zorder', 10)
    if tdim > 1:
        kwargs.setdefault('s', 15)
        pl.scatter(dofs_coordinates[:,0], dofs_coordinates[:,1], **kwargs)
    else:
        pl.ylim((-0.1, 0.1))
        pl.scatter(dofs_coordinates, np.zeros_like(dofs_coordinates), **kwargs)

In [ ]:
def plot_hermite_dofs(bc:DirichletBC):
    """ Plots the mesh and marks the Hermite dofs fulfilling 
    the boundary condition bc.
    """
    plot_dofs(bc.function_space(), find_hermite_boundary_dofs(bc), c='red')

In [ ]:
def get_facets_vertices(facetfun:FacetFunction, value:int) -> list:
    """ Returns the indices of all vertices in facets marked by facetfun.

    Arguments:
    ----------
        facetfun: FacetFunction.
        value: operate on facets where facetfun takes this value.
    Returns:
    --------
        A python list of indices.
    """
    assert isinstance(facetfun, dolfin.cpp.mesh.FacetFunctionSizet), "Blah"
    
    mesh = facetfun.mesh()
    ff = facetfun.where_equal(value)
    vv = set()
    for fa in facets(mesh):
        if fa.index() in ff:
            vv = vv.union(set(fa.entities(0)))
    return list(vv)